In [3]:
import duckdb
duckdb.execute("copy (select 1 as x, 2 as y, 'bob' as z) to 'data.parquet'")

In [5]:
from datafusion import SessionContext
import pyarrow as pa
import pyarrow.dataset as ds

In [6]:
ctx = SessionContext()

In [9]:
ctx.register_parquet("data2", "data.parquet")

In [10]:
ctx.sql("select * from data2").show()

DataFrame()
+---+---+-----+
| x | y | z   |
+---+---+-----+
| 1 | 2 | bob |
+---+---+-----+


In [ ]:
dummy_data = {
    "id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Charlie", "Diana"],
    "age": [25, 30, 35, 40]
}

# Create a PyArrow table
arrow_table = pa.Table.from_pydict(dummy_data)
ctx.from_arrow(arrow_table, "dummy_data1")


In [19]:
ctx.sql("select * from dummy_data1")

1,Alice,25
2,Bob,30
3,Charlie,35
4,Diana,40


In [16]:
sql = """
    select s.id, sum(t.age) as age_tot
    from dummy_data1 as s
        inner join dummy_data1 as t
            using(id)
    group by all
"""
ctx.sql(sql)

3,35
2,30
4,40
1,25


next steps:
- create an iceberg table via pyiceberg and see if datafusion can read it